# Setup

In [ ]:
pip install traker

In [2]:
from trak import TRAKer

def get_trak_matrix(
    train_dl, val_dl, model, ckpts, train_set_size, val_set_size, **kwargs
):
    if kwargs is None or kwargs.get("task") is None:
        task = "image_classification"
    else:
        task = kwargs.pop("task")

    traker = TRAKer(model=model, task=task, train_set_size=train_set_size, **kwargs)

    for model_id, checkpoint in enumerate(ckpts):
        traker.load_checkpoint(checkpoint, model_id=model_id)
        for batch in train_dl:
            batch = [x.cuda() for x in batch]
            # batch should be a tuple/list of inputs and labels
            traker.featurize(batch=batch, num_samples=batch[0].shape[0])

    traker.finalize_features()

    for model_id, checkpoint in enumerate(ckpts):
        traker.start_scoring_checkpoint(
            exp_name="test",
            checkpoint=checkpoint,
            model_id=model_id,
            num_targets=val_set_size,
        )
    for batch in val_dl:
        batch = [x.cuda() for x in batch]
        traker.score(batch=batch, num_samples=batch[0].shape[0])

    scores = traker.finalize_scores(exp_name="test")
    return scores


In [4]:
import torch
import numpy as np
from torch.nn import functional as F

class DDA:
    def __init__(
        self,
        model,
        checkpoints,
        train_dataloader,
        val_dataloader,
        group_indices,
        train_set_size=None,
        val_set_size=None,
        trak_scores=None,
        trak_kwargs=None,
        device="cuda",
    ) -> None:
        
        self.model = model
        self.checkpoints = checkpoints
        self.dataloaders = {"train": train_dataloader, "val": val_dataloader}
        self.group_indices = group_indices
        self.device = device

        if trak_scores is not None:
            self.trak_scores = trak_scores
        else:
            try:
                self.train_set_size = len(train_dataloader.dataset)
                self.val_set_size = len(val_dataloader.dataset)
            except AttributeError as e:
                print(
                    f"No dataset attribute found in train_dataloader or val_dataloader. {e}"
                )
                if train_set_size is None or val_set_size is None:
                    raise ValueError(
                        "train_set_size and val_set_size must be specified if "
                        "train_dataloader and val_dataloader do not have a "
                        "dataset attribute."
                    ) from e
                self.train_set_size = train_set_size
                self.val_set_size = val_set_size

            # Step 1: compute TRAK scores
            if trak_kwargs is not None:
                trak_scores = get_trak_matrix(
                    train_dl=self.dataloaders["train"],
                    val_dl=self.dataloaders["val"],
                    model=self.model,
                    ckpts=self.checkpoints,
                    train_set_size=self.train_set_size,
                    val_set_size=self.val_set_size,
                    **trak_kwargs,
                )
            else:
                trak_scores = get_trak_matrix(
                    train_dl=self.dataloaders["train"],
                    val_dl=self.dataloaders["val"],
                    model=self.model,
                    ckpts=self.checkpoints,
                    train_set_size=self.train_set_size,
                    val_set_size=self.val_set_size,
                )

            self.trak_scores = trak_scores

    def get_group_losses(self, model, val_dl, group_indices) -> list:
        losses = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_dl:
                outputs = model(inputs.to(self.device))
                loss = F.cross_entropy(
                    outputs, labels.to(self.device), reduction="none"
                )
                losses.append(loss)
        losses = torch.cat(losses)

        n_groups = len(set(group_indices))
        group_losses = [losses[group_indices == i].mean() for i in range(n_groups)]
        return group_losses

    def compute_group_alignment_scores(self, trak_scores, group_indices, group_losses):
        n_groups = len(set(group_indices))
        S = np.array(trak_scores)
        g = [
            group_losses[i].cpu().numpy() * S[:, np.array(group_indices) == i].mean(axis=1)
            for i in range(n_groups)
        ]
        g = np.stack(g)
        group_alignment_scores = g.mean(axis=0)
        return group_alignment_scores

    def get_debiased_train_indices(
        self, group_alignment_scores, use_heuristic=True, num_to_discard=None
    ):
        if use_heuristic:
            return [i for i, score in enumerate(group_alignment_scores) if score >= 0]

        if num_to_discard is None:
            raise ValueError("num_to_discard must be specified if not using heuristic.")

        sorted_indices = sorted(
            range(len(group_alignment_scores)),
            key=lambda i: group_alignment_scores[i],
        )
        return sorted_indices[num_to_discard:]

    def debias(self, use_heuristic=True, num_to_discard=None):
        group_losses = self.get_group_losses(
            model=self.model,
            val_dl=self.dataloaders["val"],
            group_indices=self.group_indices,
        )

        group_alignment_scores = self.compute_group_alignment_scores(
            self.trak_scores, self.group_indices, group_losses
        )
        
        debiased_train_inds = self.get_debiased_train_indices(
            group_alignment_scores,
            use_heuristic=use_heuristic,
            num_to_discard=num_to_discard,
        )

        return debiased_train_inds


In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Water Bird

In [6]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm

waterbirds_images_path = "/kaggle/input/waterbird/waterbird"
metadata_path = "/kaggle/input/waterbird/waterbird/metadata.csv"

def get_dataloader(batch_size=32, num_workers=4, split="train", shuffle=False, augment=True):
    if augment:
        transforms_pipeline = transforms.Compose(
            [
                transforms.RandomHorizontalFlip(),
                transforms.CenterCrop(178),
                transforms.Resize(128),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]
        )
    else:
        transforms_pipeline = transforms.Compose(
            [
                transforms.CenterCrop(178),
                transforms.Resize(128),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]
        )

    metadata = pd.read_csv(metadata_path)
    split_mapping = {"train": 0, "val": 1, "test": 2}
    split_label = split_mapping[split]
    selected_metadata = metadata[metadata["split"] == split_label]

    class WaterbirdsDataset(Dataset):
        def __init__(self, metadata, img_dir, transform=None):
            self.metadata = metadata
            self.img_dir = img_dir
            self.transform = transform

        def __len__(self):
            return len(self.metadata)

        def __getitem__(self, idx):
            row = self.metadata.iloc[idx]
            img_path = os.path.join(self.img_dir, row["img_filename"])
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            label = torch.tensor(row["y"], dtype=torch.long)
            return image, label

    dataset = WaterbirdsDataset(metadata=selected_metadata, img_dir=waterbirds_images_path, transform=transforms_pipeline)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    return loader, dataset

from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=1e-4, weight_decay=1e-4
)

train_loader, train_dataset = get_dataloader(batch_size=32, split="train", shuffle=True)
val_loader, val_dataset = get_dataloader(batch_size=32, split="val", shuffle=False, augment=False)

print(f"Total training images: {len(train_dataset)}")
print(f"Total validation images: {len(val_dataset)}")

num_epochs = 5
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    model.train()
    epoch_loss = 0.0
    for images, labels in tqdm(train_loader, desc="Training"):
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Training Loss: {avg_loss:.4f}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

print("Training and evaluation completed.")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 74.7MB/s]


Total training images: 4795
Total validation images: 1199

Epoch 1/5


Training: 100%|██████████| 150/150 [00:14<00:00, 10.57it/s]


Training Loss: 0.2211


Validation: 100%|██████████| 38/38 [00:03<00:00, 10.25it/s]


...

Validation Accuracy: 81.65%

Epoch 5/5


Training: 100%|██████████| 150/150 [00:07<00:00, 20.32it/s]


Training Loss: 0.0308


Validation: 100%|██████████| 38/38 [00:01<00:00, 23.10it/s]

Validation Accuracy: 82.57%
Training and evaluation completed.


In [7]:
import copy

model_before_mitigating = copy.deepcopy(model)

In [8]:
from sklearn.metrics import accuracy_score
import numpy as np

def evaluate_worst_group_accuracy(model, val_loader, group_inds, device="cuda"):
    model.eval() 
    group_preds = {i: [] for i in set(group_inds)}
    group_labels = {i: [] for i in set(group_inds)}

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(tqdm(val_loader, desc="Evaluating WGA")):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = batch_idx * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_inds[batch_start:batch_end]

            for i, group in enumerate(batch_groups):
                group_preds[group].append(preds[i])
                group_labels[group].append(labels.cpu().numpy()[i]) 

    group_accuracies = {}
    for group in group_preds.keys():
        if len(group_preds[group]) == 0 or len(group_labels[group]) == 0:
            group_accuracies[group] = 0.0
            continue

        preds = np.array(group_preds[group])
        truths = np.array(group_labels[group])
        group_accuracies[group] = accuracy_score(truths, preds)

    for group, acc in group_accuracies.items():
        print(f"Group {group} Accuracy: {acc:.4f}")

    worst_group_accuracy = min(group_accuracies.values())
    return worst_group_accuracy, group_accuracies

In [9]:
import pandas as pd

metadata = pd.read_csv(metadata_path)
print(metadata.size)

def define_subgroups(row):
    if row["place"] == 0:
        return "land-background"
    elif row["place"] == 1:
        return "water-background"

metadata["subgroup"] = metadata.apply(define_subgroups, axis=1)

subgroup_mapping = {name: i for i, name in enumerate(metadata["subgroup"].unique())}
metadata["group_index"] = metadata["subgroup"].map(subgroup_mapping)

print("Subgroup Distribution:")
print(metadata["subgroup"].value_counts())

val_metadata = metadata[metadata["split"] == 1]  
group_inds = val_metadata['subgroup'].map(subgroup_mapping).values
group_labels = val_metadata["group_index"].values
print(f"Sample Group Indices (Validation): {group_labels[:10]}")

# val_indices = val_loader.dataset.indices  
# group_labels = attributes.loc[val_indices, "group_index"].values
# group_inds = list(group_labels)

70728
Subgroup Distribution:
subgroup
land-background     7051
water-background    4737
Name: count, dtype: int64
Sample Group Indices (Validation): [0 1 0 0 1 0 1 1 0 1]


In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

def evaluate_group_accuracies(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_preds = {g: [] for g in set(group_labels)}
    group_truths = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating Group Accuracies")):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                group_preds[group].append(preds[j])
                group_truths[group].append(labels.cpu().numpy()[j])

    group_accuracies = {}
    for group in group_preds:
        if len(group_preds[group]) == 0:
            group_accuracies[group] = 0.0
        else:
            preds = np.array(group_preds[group])
            truths = np.array(group_truths[group])
            group_accuracies[group] = accuracy_score(truths, preds)

    for group, acc in group_accuracies.items():
        print(f"Group {group} Accuracy: {acc:.4f}")
    
    return group_accuracies

def evaluate_demographic_parity(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_pprs = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, _) in enumerate(tqdm(val_loader, desc="Evaluating Demographic Parity")):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(preds)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                group_pprs[group].append(preds[j])

    ppr_disparities = {}
    for group in group_pprs:
        group_positive_rate = np.mean(group_pprs[group])
        ppr_disparities[group] = group_positive_rate

    for group, ppr in ppr_disparities.items():
        print(f"Group {group} PPR: {ppr:.4f}")
    
    return ppr_disparities

def evaluate_equal_opportunity(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_tprs = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating Equal Opportunity")):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                tp = (preds[j] == 1 and labels[j].cpu().numpy() == 1)
                actual_positive = labels[j].cpu().numpy() == 1
                group_tprs[group].append(tp / (actual_positive + 1e-8))

    tpr_disparities = {}
    for group in group_tprs:
        tpr_disparities[group] = np.mean(group_tprs[group])

    for group, tpr in tpr_disparities.items():
        print(f"Group {group} TPR: {tpr:.4f}")
    
    return tpr_disparities

def evaluate_equalized_odds(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_tprs = {g: [] for g in set(group_labels)}
    group_fprs = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating Equalized Odds")):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                tp = (preds[j] == 1 and labels[j].cpu().numpy() == 1)
                fp = (preds[j] == 1 and labels[j].cpu().numpy() == 0)
                actual_positive = labels[j].cpu().numpy() == 1
                actual_negative = labels[j].cpu().numpy() == 0

                group_tprs[group].append(tp / (actual_positive + 1e-8))
                group_fprs[group].append(fp / (actual_negative + 1e-8))

    tpr_disparities = {}
    fpr_disparities = {}
    for group in group_tprs:
        tpr_disparities[group] = np.mean(group_tprs[group])
        fpr_disparities[group] = np.mean(group_fprs[group])

    for group in group_tprs:
        print(f"Group {group} TPR: {tpr_disparities[group]:.4f}, FPR: {fpr_disparities[group]:.4f}")
    
    return tpr_disparities, fpr_disparities

In [11]:
wga, group_accuracies = evaluate_worst_group_accuracy(model_before_mitigating, val_loader, group_labels)
dp_rates = evaluate_demographic_parity(model_before_mitigating, val_loader, group_labels)
eo_tprs = evaluate_equal_opportunity(model_before_mitigating, val_loader, group_labels)
tpr_disparities, fpr_disparities = evaluate_equalized_odds(model_before_mitigating, val_loader, group_labels)

Evaluating WGA: 100%|██████████| 38/38 [00:01<00:00, 20.18it/s]


Group 0 Accuracy: 0.7629
Group 1 Accuracy: 0.8883


Evaluating Demographic Parity: 100%|██████████| 38/38 [00:01<00:00, 22.65it/s]


Group 0 PPR: 0.4190
Group 1 PPR: 0.1300


Evaluating Equal Opportunity: 100%|██████████| 38/38 [00:01<00:00, 21.33it/s]


Group 0 TPR: 0.2020
Group 1 TPR: 0.1200


Evaluating Equalized Odds: 100%|██████████| 38/38 [00:01<00:00, 21.89it/s]

Group 0 TPR: 0.2020, FPR: 0.2170
Group 1 TPR: 0.1200, FPR: 0.0100


# Debiasing with D3M

In [12]:
print('YOYO')
ckpts = [model_before_mitigating.state_dict()]
dda = DDA(model_before_mitigating, ckpts, train_loader, val_loader, group_inds)

YOYO


Finalizing scores for all model IDs..: 100%|██████████| 1/1 [00:00<00:00, 18.41it/s]


In [31]:
# group_alignment_scores = dda.compute_group_alignment_scores(
#     dda.trak_scores, dda.group_indices, dda.get_group_losses(
#         dda.model, dda.dataloaders["val"], dda.group_indices
#     )
# )
# print("Group Alignment Scores Stats:")
# print(f"Mean: {np.mean(group_alignment_scores):.4f}")
# print(f"Min: {np.min(group_alignment_scores):.4f}")
# print(f"Max: {np.max(group_alignment_scores):.4f}")

Group Alignment Scores Stats:
Mean: -0.0001
Min: -0.1323
Max: 0.0777


In [14]:
debiased_inds = dda.debias(use_heuristic=False, num_to_discard=1500)
# debiased_inds = dda.debias(use_heuristic=True)
len(debiased_inds)

3295

In [15]:
import copy

deep_copy_model = copy.deepcopy(model_before_mitigating)

# Machine Unlearning

In [16]:
all_train_indices = np.arange(len(train_loader.dataset))

harmful_indices = np.setdiff1d(all_train_indices, debiased_inds)

print(f"Total training samples: {len(all_train_indices)}")
print(f"Debiased indices count: {len(debiased_inds)}")
print(f"Harmful indices count: {len(harmful_indices)}")

Total training samples: 4795
Debiased indices count: 3295
Harmful indices count: 1500


In [17]:
def remove_influence(model, dataloader, harmful_indices, factor, device="cuda"):
    model.eval()
    harmful_dataset = torch.utils.data.Subset(dataloader.dataset, harmful_indices)
    harmful_loader = torch.utils.data.DataLoader(harmful_dataset, batch_size=1)

    for inputs, labels in harmful_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        loss = torch.nn.functional.cross_entropy(outputs, labels)
        grads = torch.autograd.grad(loss, model.parameters(), retain_graph=True)

        with torch.no_grad():
            for param, grad in zip(model.parameters(), grads):
                param -= grad * factor

    return model

results ={'factor':[], 'model':[], 'min':[], 'max':[], 'gap':[]}
factors = np.linspace(0.0001, 0.001, 10)

for factor in factors:
    newdeepmodel = copy.deepcopy(deep_copy_model)
    m = remove_influence(newdeepmodel, train_loader, harmful_indices, factor, device="cuda")
    wga, group_accs = evaluate_worst_group_accuracy(m, val_loader, group_inds, device="cuda")
    current_gap = (max(group_accs.values()) - wga)
    results['model'].append(m)
    results['min'].append(wga)
    results['max'].append(max(group_accs.values()))
    results['gap'].append(current_gap)
    results['factor'].append(factor)

Evaluating WGA: 100%|██████████| 38/38 [00:01<00:00, 22.24it/s]


Group 0 Accuracy: 0.8481
Group 1 Accuracy: 0.8567


Evaluating WGA: 100%|██████████| 38/38 [00:01<00:00, 21.95it/s]


Group 0 Accuracy: 0.8648
Group 1 Accuracy: 0.8500


Evaluating WGA: 100%|██████████| 38/38 [00:01<00:00, 23.76it/s]


...

Group 0 Accuracy: 0.8548
Group 1 Accuracy: 0.7967


Evaluating WGA: 100%|██████████| 38/38 [00:01<00:00, 23.77it/s]


Group 0 Accuracy: 0.7780
Group 1 Accuracy: 0.7783


Evaluating WGA: 100%|██████████| 38/38 [00:01<00:00, 24.01it/s]

Group 0 Accuracy: 0.7780
Group 1 Accuracy: 0.7783


In [18]:
import pandas as pd

df = pd.DataFrame(results).sort_values('factor')
df

,factor,model,min,max,gap
0,0.0001,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.848080,0.856667,0.008587
1,0.0002,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.850000,0.864775,0.014775
2,0.0003,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.831386,0.850000,0.018614
3,0.0004,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.841402,0.850000,0.008598
4,0.0005,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.834725,0.866667,0.031942
5,0.0006,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.811667,0.861436,0.049769
6,0.0007,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.790000,0.843072,0.053072
7,0.0008,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.796667,0.854758,0.058091
8,0.0009,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.777963,0.778333,0.000370
9,0.0010,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.777963,0.778333,0.000370


Now, it's time to investigate what are the best approaches to machine unlearning and how can we formulate that.

What are the other approaches to machine unlearning?

# Fair Pruning

In [ ]:
import torch
import copy

def fair_pruning(model, dataloader, harmful_indices, threshold=0.01, device="cuda"):
    model.eval()
    pruned_model = copy.deepcopy(model)
    harmful_dataset = torch.utils.data.Subset(dataloader.dataset, harmful_indices)
    harmful_loader = torch.utils.data.DataLoader(harmful_dataset, batch_size=1)

    parameter_gradients = []
    for inputs, labels in tqdm(harmful_loader, desc="Calculating Gradients"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = pruned_model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        grads = torch.autograd.grad(loss, pruned_model.parameters(), retain_graph=True)
        parameter_gradients.append([grad.clone() for grad in grads])

    with torch.no_grad():
        for param, grads in zip(pruned_model.parameters(), zip(*parameter_gradients)):
            mean_grad = torch.mean(torch.stack(grads), dim=0)
            param[torch.abs(mean_grad) < threshold] = 0.0

    return pruned_model

pruned_model = fair_pruning(model_before_mitigating, train_loader, harmful_indices, threshold=0.01)

wga, group_accs = evaluate_worst_group_accuracy(pruned_model, val_loader, group_inds, device="cuda")

# Differentially Private Influence Functions for Unlearning

In [17]:
import torch
import numpy as np
import copy

def dp_influence_unlearning(model, dataloader, harmful_indices, epsilon=1.0, delta=1e-5, device="cuda"):
    model.eval()
    updated_model = copy.deepcopy(model)
    harmful_dataset = torch.utils.data.Subset(dataloader.dataset, harmful_indices)
    harmful_loader = torch.utils.data.DataLoader(harmful_dataset, batch_size=1)

    sensitivity = 1.0 / len(harmful_loader)
    noise_scale = sensitivity * np.sqrt(2 * np.log(1.25 / delta)) / epsilon

    for inputs, labels in tqdm(harmful_loader, desc="Applying DP Influence Unlearning"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = updated_model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        grads = torch.autograd.grad(loss, updated_model.parameters(), retain_graph=True)

        with torch.no_grad():
            for param, grad in zip(updated_model.parameters(), grads):
                noise = torch.normal(mean=0, std=noise_scale, size=grad.shape, device=device)
                param -= (grad + noise)

    return updated_model

dp_model = dp_influence_unlearning(model_before_mitigating, train_loader, harmful_indices, epsilon=1.0, delta=1e-5)
wga, group_accs = evaluate_worst_group_accuracy(dp_model, val_loader, group_inds, device="cuda")

Evaluating WGA: 100%|██████████| 38/38 [00:01<00:00, 20.29it/s]

Group 0 Accuracy: 0.7780
Group 1 Accuracy: 0.7783
